Function for creating shorts

In [9]:
import os
import tempfile
from moviepy.editor import VideoFileClip, TextClip, CompositeVideoClip, concatenate_audioclips, AudioFileClip, CompositeAudioClip
import pyttsx3

def split_text_into_parts(text, max_length=30):
    words = text.split()
    parts = []
    current_part = []
    current_length = 0
    
    for word in words:
        if current_length + len(word) > max_length:
            parts.append(' '.join(current_part))
            current_part = [word]
            current_length = len(word)
        else:
            current_part.append(word)
            current_length += len(word) + 1
    
    if current_part:
        parts.append(' '.join(current_part))
    
    return parts

def text_to_speech_pyttsx3(text, output_path):
    engine = pyttsx3.init()
    engine.setProperty('rate', 180)  # Adjust as needed
    engine.setProperty('volume', 0.9)  # Adjust as needed
    engine.setProperty('voice', 'com.apple.speech.synthesis.voice.samantha.premium')
    engine.save_to_file(text, output_path)
    engine.runAndWait()

def create_short_with_narration(video_path, text, output_folder, target_duration=57):
    video_clip = VideoFileClip(video_path)

    # Crop the video to a 9:16 aspect ratio if necessary
    target_aspect_ratio = 9.0 / 16.0
    if video_clip.w / video_clip.h > target_aspect_ratio:
        new_width = int(video_clip.h * target_aspect_ratio)
        crop_x = (video_clip.w - new_width) // 2
        video_clip = video_clip.crop(x1=crop_x, width=new_width)
    
    text_parts = split_text_into_parts(text)
    narration_clips = []
    text_clips = []
    
    # Create temporary narration and text clips
    for part in text_parts:
        with tempfile.NamedTemporaryFile(delete=True, suffix='.wav') as temp_audio:
            text_to_speech_pyttsx3(part, temp_audio.name)
            narration_clip = AudioFileClip(temp_audio.name)
            narration_clips.append(narration_clip)
            
            # Assume the duration of each text part is the same as its narration clip
            text_clip = TextClip(part, fontsize=40, color='white', font='Arial-Bold',align='center', size=(video_clip.w, 100))
            text_clip = text_clip.set_duration(narration_clip.duration)
            text_clip = text_clip.set_position('center').set_duration(narration_clip.duration)
            text_clips.append(text_clip)

    # Calculate total narration duration and adjust video duration accordingly
    total_narration_duration = sum([clip.duration for clip in narration_clips])
    if total_narration_duration > target_duration:
        print("Warning: Total narration exceeds target duration. Consider splitting text.")
    
    # Loop or trim the video to match the target duration
    if video_clip.duration < target_duration:
        video_clip = video_clip.loop(duration=target_duration)
    else:
        video_clip = video_clip.subclip(0, target_duration)
    
    # Set start times for text and narration clips to sequentially appear
    start_time = 0
    for i, clip in enumerate(narration_clips):
        narration_clips[i] = clip.set_start(start_time)
        text_clips[i] = text_clips[i].set_start(start_time)
        start_time += clip.duration
    
    # Combine narration clips into a single audio track and mix with original audio
    combined_narration = concatenate_audioclips(narration_clips)
    mixed_audio = CompositeAudioClip([video_clip.audio, combined_narration])
    video_clip.audio = mixed_audio
    
    # Create final clip with video and overlaid text
    final_clip = CompositeVideoClip([video_clip] + text_clips, size=video_clip.size).set_duration(target_duration)
    
    # Output
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)
    output_filename = os.path.basename(video_path).replace(".mp4", "_narrated.mp4")
    output_path = os.path.join(output_folder, output_filename)
    
    final_clip.write_videofile(output_path, codec="libx264", audio_codec="aac")


# Example usage
text = "In Python, classes act as blueprints for creating objects. They define a set of attributes (variables that hold data) and methods (functions that define behaviors) that their objects will embody. A class encapsulates data for the object it creates, ensuring a modular and organized approach to programming. When a class is defined, it doesn’t create an actual object but rather specifies what attributes and methods the objects of that class should have. Objects are instances of classes; when you instantiate a class, you’re creating a unique object with its own specific state based on the class's blueprint. This allows for individual objects to possess their own characteristics and behaviors, enabling programmers to model real-world scenarios in a structured and intuitive way. Python's class and object mechanism facilitate data abstraction, encapsulation, inheritance, and polymorphism, making it powerful for developing complex applications."
create_short_with_narration("Clips_for_edit/1 hour 20 minutes of relaxing Minecraft Parkour (Nostalgia, Scenery, No Ads)_3.mp4", text, "Final_product")


Moviepy - Building video Final_product/1 hour 20 minutes of relaxing Minecraft Parkour (Nostalgia, Scenery, No Ads)_3_narrated.mp4.
MoviePy - Writing audio in 1 hour 20 minutes of relaxing Minecraft Parkour (Nostalgia, Scenery, No Ads)_3_narratedTEMP_MPY_wvf_snd.mp4


MoviePy - Done.
Moviepy - Writing video Final_product/1 hour 20 minutes of relaxing Minecraft Parkour (Nostalgia, Scenery, No Ads)_3_narrated.mp4



Moviepy - Done !
Moviepy - video ready Final_product/1 hour 20 minutes of relaxing Minecraft Parkour (Nostalgia, Scenery, No Ads)_3_narrated.mp4


Managing shorts and posting them on social media

In [ ]:
import os
from dotenv import load_dotenv
from pyyoutube import Api as YouTubeApi
from tiktokpy import TikTokPy
from instabot import Bot

# Load environment variables from .env file
load_dotenv()

# Get API keys from environment variables
youtube_api_key = os.getenv("YOUTUBE_API_KEY")
tiktok_api_key = os.getenv("TIKTOK_API_KEY")
instagram_username = os.getenv("INSTAGRAM_USERNAME")
instagram_password = os.getenv("INSTAGRAM_PASSWORD")

# Connect to YouTube API
youtube_api = YouTubeApi(api_key=youtube_api_key)

# Connect to TikTok API
tiktok_api = TikTokPy(api_key=tiktok_api_key)

# Connect to Instagram API
instagram_bot = Bot()
instagram_bot.login(username=instagram_username, password=instagram_password)

# Get the path of the short video file
short_video_path = "Final_product/short_video.mp4"

# Post the short video on YouTube
youtube_api.upload_video(file_path=short_video_path, title="Short Video", description="Check out this short video!")

# Post the short video on TikTok
tiktok_api.upload_video(file_path=short_video_path, caption="Check out this short video!")

# Post the short video on Instagram
instagram_bot.upload_photo(photo=short_video_path, caption="Check out this short video!")


Things to install if you want to run the code:

In [ ]:

#pip install gtts
#pip install moviepy


Creating database with 60 seconds clip from a longer video yay

In [ ]:
import os
#os.environ["IMAGEIO_FFMPEG_EXE"] = "/opt/homebrew/lib/python3.11/site-packages/ffmpeg"
from moviepy.editor import VideoFileClip


def Cut_Into_Shorts(directory_with_long_Videos, directory_with_shorts):
    # Get a list of video files in the directory
    video_files = [f for f in os.listdir(directory_with_long_Videos) if f.endswith('.mp4')]
    
    # Iterate over each video file
    for video_file in video_files:
        # Get the full path of the video file
        video_path = os.path.join(directory_with_long_Videos, video_file)
        
        # Create a VideoFileClip object
        video_clip = VideoFileClip(video_path)
        
        # Calculate the duration of the video clip in seconds
        video_duration = video_clip.duration
        
        # Calculate the number of 60-second clips
        num_clips = int(video_duration // 60)
        
        # Iterate over each 60-second clip
        for i in range(num_clips):
            # Calculate the start and end time of the clip
            start_time = i * 60
            end_time = (i + 1) * 60
            
            # Extract the clip from the video
            clip = video_clip.subclip(start_time, end_time)
            
            # Generate the output file name
            output_file = f"{video_file.split('.')[0]}_{i+1}.mp4"
            
            # Save the clip to the output directory
            clip.write_videofile(os.path.join(directory_with_shorts, output_file))
        
        # Close the video clip object
        video_clip.close()

# Example usage
Cut_Into_Shorts("Raw_material", "Clips_for_edit")


test

In [3]:
import os
import tempfile
from moviepy.editor import VideoFileClip, TextClip, CompositeVideoClip, concatenate_audioclips, AudioFileClip, CompositeAudioClip
import pyttsx3
engine = pyttsx3.init()
voices = engine.getProperty('voices')
engine.setProperty('voice', 'com.apple.speech.synthesis.voice.samantha.premium')
engine.setProperty('rate', 180)  # Adjust as needed
engine.setProperty('volume', 0.9)  # Adjust as needed
engine.say("Hello World! This is a test.")
engine.runAndWait()
"""for voice in voices:
    if ['en_US'] == voice.languages:
        print(voice.languages)
        print(voice, voice.id)
        engine.setProperty('voice', voice.id)
        engine.setProperty('rate', 180)  # Adjust as needed
        engine.setProperty('volume', 0.9)  # Adjust as needed
        engine.say("Hello World! This is a test.")
        engine.runAndWait()
        engine.stop()
    else:
        pass
"""


'for voice in voices:\n    if [\'en_US\'] == voice.languages:\n        print(voice.languages)\n        print(voice, voice.id)\n        engine.setProperty(\'voice\', voice.id)\n        engine.setProperty(\'rate\', 180)  # Adjust as needed\n        engine.setProperty(\'volume\', 0.9)  # Adjust as needed\n        engine.say("Hello World! This is a test.")\n        engine.runAndWait()\n        engine.stop()\n    else:\n        pass\n'